In [6]:
%matplotlib inline
import pyspeckit
import numpy as np
from astropy import units as u
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.convolution import convolve, Box1DKernel
from c_normalize import c_normalize
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
plt.style.use('seaborn-paper')

In [7]:
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [14]:
# Select Datasets and extract equivalent widths
spectrum = glob.glob("data/ew_known/tame_inputs/*wavsoln.fits")
by_hand = glob.glob("tame-1.1.0/*.ew")
lc_range = np.arange(0.95,0.995,0.0001)
len_range = 10


d = {'Spectrum File': spectrum, 'By-Hand File': by_hand}
disp_df = pd.DataFrame(data=d)

lines = pd.read_csv(by_hand[0], header=None, delim_whitespace=True)[0].tolist()

In [19]:

width = 10 #Distance from the line on both sides to sample the local continuum from
gauss_amp = -0.2
gauss_width = 0.15
gauss_centhresh = 0.2

pars_dict={'7112.17':(10, -0.3, 0.15, 0.2)}

for line in lines:
    percent_difference = False
    diff = []
    for row in disp_df.iterrows():
        eqw_file, spec_file = row[1]
        eqw_df = pd.read_csv(eqw_file, header=None, delim_whitespace = True)
        if line not in eqw_df[0].tolist():
            print("{} Line Missing: {}".format(spec_file,line))
            continue
        mask = eqw_df[0] == line
        byhand_eqw = eqw_df[4][mask.tolist()].tolist()[0]
        if byhand_eqw == 0.:
            continue
    
        s_hdu = fits.open(spec_file)
    
        # Grab the flux and wavelength arrays from the spectrum
        s_data = s_hdu[1].data
        s_flux = s_data['FLUX']
        smoothed_flux = convolve(s_flux, Box1DKernel(5)) # Smooth the flux for more reliable continuum fitting
        s_flux = smoothed_flux
        s_wav = s_data['WAVEL']

        # Specify up a specific line to measure the EQW for, and specify the wavelength range to sample for the local continuum
        lim_l = line - width
        lim_r = line + width

        # Mask the flux and wavelength arrays based on the sampled wavelength range
        wav_mask = (s_wav > lim_l) & (s_wav < lim_r)
        s_flux = s_flux[wav_mask]
        s_wav = s_wav[wav_mask]

        # Normalize the spectrum with the local continuum to 1.0, c_normalize returns the normalized spectrum in norm, but also the local continuum fit in yfit if desired
        yfit, norm, _ = c_normalize(s_flux, s_wav, median_replace=False, cheby=True, low_cut = 0.99)
        # Load the normalized spectrum into a pyspeckit.Spectrum object
        sp = pyspeckit.Spectrum(data=norm, xarr=s_wav * u.AA)

        #Get around pyspeckits reliance on a generated baseline and just generate basespec as a ones array (continuum is always one in normalized case)
        sp.baseline.basespec = np.ones(len(s_wav))

        #Fit a gaussian to the line, this may be too simplistic for blended lines
        with suppress_stdout(): #Suppress some annoying info messages that don't translate well to automation
            sp.specfit(fittype='gaussian', guesses = [gauss_amp, line, gauss_width], exclude= [0,line-gauss_centhresh, line+gauss_centhresh, line+5000])

        #Measure the Equivalent Width of the gaussian line fit against the normalized baseline
        EQW = sp.specfit.EQW(plot=False, continuum_as_baseline=True, xmin = 0, xmax = len(norm))
        EQW = EQW*1000 #mA
        if percent_difference:
            ew_diff = ((EQW-byhand_eqw)/byhand_eqw) * 100 #Percent Difference
        else:
            ew_diff = EQW-byhand_eqw #mA Difference
        if ew_diff < 1000: #Try to cut out some of these crazy fits
            diff.append(ew_diff)
    if percent_difference:
        symbol = "%"
    else:
        symbol = " mA"
    print("{} Statistics".format(line))
    print("===============")

    print("Median Difference: {}{}".format(np.median(diff),symbol))
    print("Standard Deviation of Difference: {}{}".format(np.std(diff), symbol))
    print("~95% within {}{}".format(np.round(2*np.std(diff), 2), symbol))

5307.36 Statistics
Median Difference: 1.9301600540973212 mA
Standard Deviation of Difference: 3.8983035810374926 mA
~95% within 7.8 mA
5322.04 Statistics
Median Difference: 4.256415038045809 mA
Standard Deviation of Difference: 6.304528320013537 mA
~95% within 12.61 mA
5466.99 Statistics
Median Difference: 2.2849277943120967 mA
Standard Deviation of Difference: 3.7194371109548294 mA
~95% within 7.44 mA
5491.83 Statistics
Median Difference: -4.795721403959924 mA
Standard Deviation of Difference: 5.482889932559583 mA
~95% within 10.97 mA
5522.45 Statistics
Median Difference: 3.5505787396837434 mA
Standard Deviation of Difference: 5.54351882495493 mA
~95% within 11.09 mA
5536.58 Statistics
Median Difference: 4.646521716730918 mA
Standard Deviation of Difference: 5.3470300392495504 mA
~95% within 10.69 mA
5539.28 Statistics
Median Difference: 3.0193999880130704 mA
Standard Deviation of Difference: 6.210291375585676 mA
~95% within 12.42 mA
5560.21 Statistics
Median Difference: 0.10536889519